In [ ]:
!python3 -m spacy download en

In [ ]:
!pip install pyLDAvis

In [ ]:
!pip install gensim 

In [16]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis

import pyLDAvis.gensim_models as gensimvis
#from pyLDAvis.gensim   # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
from os import listdir
import glob
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

path = '/home/mikhail/it-academy/модуль 6/bbc/'
dfs = []
for i, dirname in enumerate(listdir(path) ):
    filenames = glob.glob(path + dirname + '/*.txt')
    for filename in filenames:
         text = open(filename, mode="rb").read()
         dfs.append((text, dirname))


df = pd.DataFrame(dfs, columns=['text', 'target'])

labels=df.target.unique()
id2label={i:l for i,l in enumerate(df.target.unique())}
label2id={l:i for i,l in enumerate(labels)}
df["labels"]=df.target.map(lambda x: label2id[x])


df.text = df.text.apply(lambda x: str(x)[1:])  #удаляем символ b, который появился в начале строки после чтения из файла
df.text = df.text.apply(lambda x: str(x).replace('\\n',' '))  #удаляем все переносы строк
df.text = df.text.apply(lambda x: str(x).replace('\\', '')) #удаляем апострофы


data = df.text.values.tolist()
#data = [re.sub('\s+', ' ', sent) for sent in data]   
data = [re.sub("\'", "", sent) for sent in data]  #почему-то не все апострофы удалились, еще раз удалим те, что остались

#print(data[:1])

#Токенизация слов и очистка текста от знаков пунктуации
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  #удаляем пунктуации и разбиваем документ на токены
data_words = list(sent_to_words(data))
#print(data_words[:1])


# Создание биграмм и триграмм
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
#print(trigram_mod[bigram_mod[data_words[0]]])



def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Удалим стоп-слова, сделаем биграммы и лемматизируем
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized[:1])


# Создадим словарь и корпус.
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=len(df.target.unique()), 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

print(lda_model.print_topics())

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mikhail/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[(0, '0.045*"film" + 0.020*"music" + 0.013*"show" + 0.011*"song" + 0.010*"new" + 0.010*"year" + 0.009*"album" + 0.009*"say" + 0.008*"tv" + 0.007*"movie"'), (1, '0.042*"say" + 0.011*"make" + 0.009*"go" + 0.009*"people" + 0.007*"get" + 0.006*"want" + 0.006*"think" + 0.006*"also" + 0.005*"work" + 0.005*"new"'), (2, '0.026*"good" + 0.020*"award" + 0.014*"star" + 0.011*"year" + 0.010*"actor" + 0.009*"play" + 0.009*"take" + 0.009*"director" + 0.009*"include" + 0.008*"win"'), (3, '0.032*"year" + 0.014*"say" + 0.014*"rise" + 0.011*"fall" + 0.009*"market" + 0.009*"sale" + 0.009*"month" + 0.009*"last" + 0.009*"price" + 0.009*"economy"'), (4, '0.031*"say" + 0.015*"company" + 0.013*"firm" + 0.008*"share" + 0.006*"business" + 0.006*"government" + 0.006*"profit" + 0.006*"also" + 0.005*"deal" + 0.005*"case"')]

Coherence Score:  0.44633815951916117


/home/mikhail/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import re

Вывод

1 - politics
2 - entertainment
3 - tech
4 - sport
5 - business